 # projeto sem objetivo pre definido

* primeiro fazer coleta dos dados para armazenamento 
*  sera feito coleta atraves de web scraping
* site: https://www.zapimoveis.com.br
* limpeza de dados
* talvez crie um banco de dados para mostrar a criacao de um banco de dados
* talvez pode ser feito uma coleta diaria durante um tempo para saber qto tempo um imovel fica disponivel para venda, qual o tempo medio de espera para ser vendido, qual o preço medio dos  imoveis mais vendidos 

apos coleta sera feita uma analise exploratoria para saber o que sera possivel de analisar, como por exemplo, valor media das casa, valor medio por regiao, a media de comodos, regioes  e comodos afetam os preços, qual tem mais impacto...?

tecnologia esperadas q serao usadas

* python
    * selenium ou beutiful soup para coleta de dados
    * pandas para exploraçao e limpeza

* power bi
    * para apresentaçao final

*github para fazer os historicos de evoluçao diaria e ter noçao de qto tempo levou para fazer o projeto (nao estou me obrigando a ter prazo, tenho outro trabalho entao sera feito nas horas disponiveis sem afetar outros compromissos)






In [1]:
#importando as bibliotecas necessarias
import pandas as pd
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [2]:
# explarando o site com filtro de casa para venda e na ordem de maiores valores
driver_casa = webdriver.Chrome()
driver_casa.get('https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=1&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda')
driver_casa.maximize_window()
original_window = driver_casa.current_window_handle

In [3]:
#pegando os anuncios das casas
anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

In [4]:
#conferindo a quantidade de elementos 
len(anuncios)

24

entrando no anuncio para verificar o que é possivel extrair 

fazendo uma breve verificacao em algums anuncio vi q os valores nao correspondem ao valor real,
 entao apos a coleta de dados sera necessario ainda uma limpeza 

 aqui ja da para extrair :

 valor do imovel
 
 condominio
 
 iptu
 
 area
 
 quantidade de banheiro
 
 quantidade de quartos
 
 quantidade de vagas na garagem
 
 rua, numero e bairro (cidade e estado sao sempre iguais, ja q é uma pesquisa de imoveis de sao jose dos campos)
 
 se esta por imobiliaria ou nao
 
 extrair tambem as informaçoes das descriçoes para complentar alguma informaçao
 
 a descricao tbm pode estar como caracteristica
 
 quantidade de andares
    

In [19]:
def verifica_aba(aba, anuncio):
    while len(aba.window_handles) > 1:
        aba.switch_to.window(aba.window_handles[1])
        aba.close()
        aba.switch_to.window(aba.window_handles[0]) 
    anuncio.click()
    sleep(1)
    aba.switch_to.window(aba.window_handles[1]) 
    

def pega_valores(pagina, anuncio):
    
    if len(pagina.window_handles) > 1:
        print(pagina.current_window_handle)
       # verifica_aba(pagina, anuncio)
    else:
        pagina.switch_to.window(pagina.window_handles[0]) 
        anuncio.click()
        sleep(1)
        pagina.switch_to.window(pagina.window_handles[1]) 

    endereco= pagina.find_element(By.CLASS_NAME, 'info__map-link ')   

    preco = pagina.find_element(By.CLASS_NAME, 'price__item--main')
    
    condominio = pagina.find_elements(By.CLASS_NAME, 'price__item')[0]
    
    iptu = pagina.find_elements(By.CLASS_NAME, 'price__item')[1]
    
    comodos = pagina.find_elements(By.CLASS_NAME, 'feature__item')
    lista_comodos = [comodo.text for comodo in comodos]
    
    caracteristicas = pagina.find_elements(By.CLASS_NAME, 'amenities__list-item')
    lista_caracteristicas = [caractistica.text for caractistica in caracteristicas]
    
    obserevacoes = pagina.find_element(By.CLASS_NAME, 'amenities__base-description')
    
    #pagina.close()
    #pagina.switch_to.window(aba.window_handles[0]) 

    return endereco, preco, condominio, iptu, lista_comodos, lista_caracteristicas, obserevacoes

In [6]:
# window_handles eh uma lista de todas as guias q estao abertas, usar esse for pra saber quais os nomes das guias
for window_handle in driver_casa.window_handles:
    print(window_handle)

CDwindow-AE6559BD6DD6DC7B86DE5F8618FEADF1


In [7]:
driver_casa.current_window_handle

'CDwindow-AE6559BD6DD6DC7B86DE5F8618FEADF1'

In [8]:
#driver_casa.switch_to.window(driver_casa.window_handles[0])

In [18]:
dados = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Area', 'Quartos',
                                                 'Vagas_Garagem', 'Banheiros', 'Caracteristica', 'Obserevacoes'])
for i in range(len(anuncios)):
    endereco, preco, condominio, iptu, comodos, caracteristicas, obserevacoes = pega_valores(driver_casa, anuncios[i])
    caracteristica = [i.text for i in caracteristicas]
    dic_dados = {'Endereço':[endereco.text],
             'Preço' : [preco.text],
             'Condominio': [condominio.text],
             'Iptu': [iptu.text],
             'Area': [comodos[4]],
             'Quartos' : [comodos[5]],
             'Vagas_Garagem' : [comodos[6]],
             'Banheiros' : [comodos[7]],
             'Caracteristica' : [caracteristica],
             'Obserevacoes' : [obserevacoes.text]}
    dados.append(dic_dados)

C:\Users\jpnun\AppData\Local\Temp\ipykernel_1740\1204797429.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados.append(dic_dados)


TypeError: Can only append a dict if ignore_index=True

In [10]:
print(f'Endereço = {endereco.text}')
print(f'preco = {preco.text}')
print(f'condominio = {condominio.text}')
print(f'iptu = {iptu.text}')
print(f'lista_comodos = {comodos[4]}')
print(f'obserevacoes = {obserevacoes.text}')


Endereço = Rua Coronel Manoel Martins Júnior, 0 - Jardim Esplanada II, São José dos Campos - SP
preco = R$ 42.000
condominio = condomínio não informado
iptu = IPTU não informado
lista_comodos = 120 m²
obserevacoes = A casa está localizado no bairro Jardim Esplanada II tem 120 metros quadrados com 2 quartos sendo 1 suite e 1 banheiro

42 mil de entrada, mais parcelas.


In [11]:
caracteristica = [i.text for i in caracteristicas]
caracteristica

[]

In [12]:

dic_dados = {'Endereço':[endereco.text],
             'Preço' : [preco.text],
             'Condominio': [condominio.text],
             'Iptu': [iptu.text],
             'Area': [comodos[4]],
             'Quartos' : [comodos[5]],
             'Vagas_Garagem' : [comodos[6]],
             'Banheiros' : [comodos[7]],
             'Caracteristica' : [caracteristica],
             'Obserevacoes' : [obserevacoes.text]}
dic_dados

{'Endereço': ['Rua Coronel Manoel Martins Júnior, 0 - Jardim Esplanada II, São José dos Campos - SP'],
 'Preço': ['R$ 42.000'],
 'Condominio': ['condomínio não informado'],
 'Iptu': ['IPTU não informado'],
 'Area': ['120 m²'],
 'Quartos': ['2 quartos'],
 'Vagas_Garagem': ['2 vagas'],
 'Banheiros': ['1 banheiro'],
 'Caracteristica': [[]],
 'Obserevacoes': ['A casa está localizado no bairro Jardim Esplanada II tem 120 metros quadrados com 2 quartos sendo 1 suite e 1 banheiro\n\n42 mil de entrada, mais parcelas.']}

In [13]:
dados = pd.DataFrame(data = dic_dados)
dados

,Endereço,Preço,Condominio,Iptu,Area,Quartos,Vagas_Garagem,Banheiros,Caracteristica,Obserevacoes
0,"Rua Coronel Manoel Martins Júnior, 0 - Jardim ...",R$ 42.000,condomínio não informado,IPTU não informado,120 m²,2 quartos,2 vagas,1 banheiro,[],A casa está localizado no bairro Jardim Esplan...


In [14]:
if driver_casa.current_window_handle != driver_casa.window_handles[0]:
    driver_casa.close()
driver_casa.switch_to.window(driver_casa.window_handles[0])
driver_casa.current_window_handle

'CDwindow-AE6559BD6DD6DC7B86DE5F8618FEADF1'

In [15]:
driver_casa.title

'Casas à venda em São José dos Campos, SP - ZAP Imóveis'

In [17]:
dados = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Area', 'Quartos',
                                                 'Vagas_Garagem', 'Banheiros', 'Caracteristica', 'Obserevacoes'])
dados

,Endereço,Preço,Condominio,Iptu,Area,Quartos,Vagas_Garagem,Banheiros,Caracteristica,Obserevacoes
